# iris

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_iris
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.tree import export_graphviz
from sklearn.tree import DecisionTreeClassifier
import warnings
#import graphviz
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
# iris 데이터 가져오기
iris = load_iris()

In [3]:
# data, target, target_name, feature_name, DESCR, 
data = iris.data
label = iris.target
columns = iris.feature_names
df = pd.DataFrame(data, columns=columns)
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [4]:
X_train, X_test, y_train, y_test, = train_test_split(data, label, shuffle=True, stratify=label,
                                                    test_size=0.2, random_state=2019)

- 모델생성

In [5]:
lr = LogisticRegression(verbose=1)

In [6]:
lr.fit(X_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


LogisticRegression(verbose=1)

- 학습한 결과 저장

In [7]:
import joblib
joblib.dump(lr,'model/iris_lr.pkl')

['model/iris_lr.pkl']

- support Vector Machine

In [8]:
# 모델 객체 생성
svc = SVC()
#svc.fit(X_train, y_train)
#y_pred_svc = svc.predict(X_test)
#accuracy_score(y_test, y_pred)

In [9]:
# 학습
svc.fit(X_train, y_train)

SVC()

In [10]:
import joblib
joblib.dump(lr,'model/iris_svm.pkl')

['model/iris_svm.pkl']

### Decision Tree

In [11]:
# 객체 모델 생성  / dtc 소문자만 가능
dtc = DecisionTreeClassifier()

In [12]:
# 학습
dtc.fit(X_train, y_train)

DecisionTreeClassifier()

In [13]:
import joblib
joblib.dump(lr,'model/iris_dt.pkl')

['model/iris_dt.pkl']

### 저장된 모델 정확도 산출

In [14]:
model_lr = joblib.load('model/iris_lr.pkl')
model_svm = joblib.load('model/iris_svm.pkl')
model_dt = joblib.load('model/iris_dt.pkl')

In [15]:
y_pred_lr = model_lr.predict(X_test)
y_pred_svm = model_svm.predict(X_test)
y_pred_dt = model_dt.predict(X_test)

In [16]:
acc_lr = accuracy_score(y_test, y_pred_lr)
acc_svm = accuracy_score(y_test, y_pred_svm)
acc_dt = accuracy_score(y_test, y_pred_dt)
print(acc_lr, acc_svm, acc_dt)

0.9666666666666667 0.9666666666666667 0.9666666666666667


### 한개의 테스트 값 예측

In [17]:
df.tail()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3
149,5.9,3.0,5.1,1.8


In [18]:
test_data = np.array([6.7, 3.0, 5.2 , 2.3]).reshape(1,4)   # 2차원을 1차원으로 reshape

In [19]:
index_lr = model_lr.predict(test_data)[0]   # [0]으로 숫자만 출력
index_svm = model_lr.predict(test_data)[0]
index_dt = model_lr.predict(test_data)[0]

In [20]:
sp_names = ['Setosa', 'Versicolor', 'Virginica']
print(sp_names[index_lr], sp_names[index_svm], sp_names[index_dt])

Virginica Virginica Virginica


## Deep learnin model

In [21]:
# 딥러닝 모델 설정
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow import keras
from tensorflow.keras.layers import Dense

model = Sequential([
    Dense(12, input_shape=(4, ), activation='relu'),
    Dense(8, activation='relu'),
    Dense(3, activation='softmax')
]) 
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 12)                60        
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 104       
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 27        
Total params: 191
Trainable params: 191
Non-trainable params: 0
_________________________________________________________________


In [22]:
# 모델 컴파일 
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [23]:
Y_eoncoded = keras.utils.to_categorical(y_train)

In [24]:
cp = ModelCheckpoint('model/iris_deep.h5', verbose=1, save_best_only=True)

In [25]:
model.fit(X_train, Y_eoncoded, validation_split=0.2, epochs=100, batch_size=30, 
          verbose=0, callbacks=[cp])


Epoch 00001: val_loss improved from inf to 3.14947, saving model to model/iris_deep.h5

Epoch 00002: val_loss improved from 3.14947 to 2.92497, saving model to model/iris_deep.h5

Epoch 00003: val_loss improved from 2.92497 to 2.71316, saving model to model/iris_deep.h5

Epoch 00004: val_loss improved from 2.71316 to 2.50962, saving model to model/iris_deep.h5

Epoch 00005: val_loss improved from 2.50962 to 2.32001, saving model to model/iris_deep.h5

Epoch 00006: val_loss improved from 2.32001 to 2.14444, saving model to model/iris_deep.h5

Epoch 00007: val_loss improved from 2.14444 to 1.98117, saving model to model/iris_deep.h5

Epoch 00008: val_loss improved from 1.98117 to 1.83289, saving model to model/iris_deep.h5

Epoch 00009: val_loss improved from 1.83289 to 1.69563, saving model to model/iris_deep.h5

Epoch 00010: val_loss improved from 1.69563 to 1.56983, saving model to model/iris_deep.h5

Epoch 00011: val_loss improved from 1.56983 to 1.45695, saving model to model/iris_


Epoch 00091: val_loss improved from 0.34915 to 0.34586, saving model to model/iris_deep.h5

Epoch 00092: val_loss improved from 0.34586 to 0.34260, saving model to model/iris_deep.h5

Epoch 00093: val_loss improved from 0.34260 to 0.33999, saving model to model/iris_deep.h5

Epoch 00094: val_loss improved from 0.33999 to 0.33835, saving model to model/iris_deep.h5

Epoch 00095: val_loss improved from 0.33835 to 0.33378, saving model to model/iris_deep.h5

Epoch 00096: val_loss improved from 0.33378 to 0.32967, saving model to model/iris_deep.h5

Epoch 00097: val_loss improved from 0.32967 to 0.32647, saving model to model/iris_deep.h5

Epoch 00098: val_loss improved from 0.32647 to 0.32290, saving model to model/iris_deep.h5

Epoch 00099: val_loss improved from 0.32290 to 0.31831, saving model to model/iris_deep.h5

Epoch 00100: val_loss improved from 0.31831 to 0.31566, saving model to model/iris_deep.h5


In [31]:
from tensorflow.keras.models import load_model
model_deep = load_model('model/iris_deep.h5')

In [32]:
model_deep.predict(test_data)[0]

array([0.04672099, 0.45200348, 0.5012755 ], dtype=float32)

In [33]:
np.argmax(model_deep.predict(test_data)[0])

2

In [29]:
#iris_deep = model_deep.predict_classes(test_data)[0]

In [30]:
#print(sp_names[iris_deep])